In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Load model
model_name = 'eryk-mazus/polka-1.1b'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

print(f"[INFO] Model '{model_name}' loaded on {device}.")

[INFO] Model 'eryk-mazus/polka-1.1b' loaded on cuda.


In [3]:
# 2. Suppose we have some riddles and a known set of valid answers.

riddles = [
    "kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem",
    "emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu",
    "największe miasto w Polsce",
]

answer_set = [
    "pasażerka",
    "miłość",
    "warszawa",
    # ... you can add more known answers if needed
]

print("Riddles:")
for idx, r in enumerate(riddles, 1):
    print(f"{idx}. {r}")
print("\nPossible Answers:", answer_set)

Riddles:
1. kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem
2. emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu
3. największe miasto w Polsce

Possible Answers: ['pasażerka', 'miłość', 'warszawa']


In [4]:
def build_few_shot_prompt(riddle_text: str, examples=None) -> str:
    """
    Creates a few-shot prompt with riddle -> answer pairs,
    then ends with the new riddle for which we want an answer.
    """
    if examples is None:
        examples = [
            # Minimal example from your instructions:
            ("kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem", "pasażerka"),
            ("emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu", "miłość")
        ]
    prompt = ""
    for ex_riddle, ex_answer in examples:
        prompt += f"Zagadka: {ex_riddle}\nOdpowiedź: {ex_answer}\n\n"
    # Add the new riddle
    prompt += f"Zagadka: {riddle_text}\nOdpowiedź:"
    return prompt

def constrained_generate_answer(
    prompt: str,
    answer_set: list[str],
    max_new_tokens: int = 20,
    temperature: float = 1.0
) -> str:
    """
    Iteratively generates tokens from 'prompt', but only allows partial sequences
    that remain a prefix of at least one valid answer in 'answer_set'.
    
    Once we match a full answer from answer_set, we stop.
    The final answer is the portion after 'Odpowiedź:'.
    """
    # 1) Encode the prompt
    input_ids = tokenizer(prompt, return_tensors='pt')['input_ids'].to(device)
    
    generated = input_ids.clone()  # we keep track of the growing sequence
    
    # We'll store the partial decoded answer (the portion after 'Odpowiedź:') as we go
    # Each iteration, we decode everything after the prompt, see if it's a prefix
    # of any valid answer
    start_len = input_ids.shape[1]  # length of the entire prompt
    eos_token_id = tokenizer.eos_token_id
    partial_answer = ""

    def is_prefix_of_answer(s: str) -> bool:
        return any(ans.startswith(s) for ans in answer_set)

    def is_full_answer(s: str) -> bool:
        return s in answer_set

    # 2) Iterative generation
    for step in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(generated)
            logits = outputs.logits[:, -1, :]  # shape: [1, vocab_size]
        
        # We'll build new logits by setting -inf for tokens that
        # don't keep partial_answer as prefix of any valid answer
        new_logits = torch.full_like(logits, float('-inf'))
        
        # We try each possible vocab token and see if it leads to a valid prefix
        for token_id in range(logits.shape[1]):
            # skip special tokens if you want
            if token_id == eos_token_id:
                # Potentially allow EOS if partial_answer is already full?
                # we'll skip it for now
                continue

            # Hypothesize appending this token
            cand_ids = torch.cat([generated[0], torch.tensor([token_id]).to(device)])
            cand_text = tokenizer.decode(cand_ids, skip_special_tokens=True)
            # The newly generated portion is everything after the prompt
            new_part = cand_text[len(prompt):].strip()

            if new_part:
                # Only keep this token if new_part is prefix of at least one answer
                if is_prefix_of_answer(new_part):
                    new_logits[0, token_id] = logits[0, token_id]
            else:
                # if new_part is empty, it's also valid as a prefix
                new_logits[0, token_id] = logits[0, token_id]

        # Apply temperature
        if temperature > 0:
            new_logits = new_logits / temperature
            probs = F.softmax(new_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        else:
            # Greedy
            next_token = torch.argmax(new_logits, dim=-1, keepdim=True)

        # Append
        generated = torch.cat([generated, next_token], dim=1)

        # Decode partial answer
        full_text = tokenizer.decode(generated[0], skip_special_tokens=True)
        partial_answer = full_text[len(prompt):].strip()

        # If partial_answer is a complete valid answer, we stop
        if is_full_answer(partial_answer):
            break

    return partial_answer

# Quick test
test_prompt = build_few_shot_prompt("największe miasto w Polsce")
print("Prompt:\n", test_prompt, "\n")

ans = constrained_generate_answer(test_prompt, ["pasażerka","miłość","warszawa"], max_new_tokens=10, temperature=0.7)
print("Generated Answer:", ans)

Prompt:
 Zagadka: kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem
Odpowiedź: pasażerka

Zagadka: emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu
Odpowiedź: miłość

Zagadka: największe miasto w Polsce
Odpowiedź: 

Generated Answer: warszawa


In [5]:
few_shot_examples = [
    ("kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem", "pasażerka"),
    ("emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu", "miłość"),
]

for riddle_text in riddles:
    # Build a few-shot prompt
    prompt = build_few_shot_prompt(riddle_text, examples=few_shot_examples)
    
    # Generate constrained answer
    answer = constrained_generate_answer(prompt, answer_set, max_new_tokens=10, temperature=0.7)
    
    print("Riddle:", riddle_text)
    print("Answer:", answer)
    print("-"*50)


Riddle: kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem
Answer: pasażerka
--------------------------------------------------
Riddle: emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu
Answer: miłość
--------------------------------------------------
Riddle: największe miasto w Polsce
Answer: warszawa
--------------------------------------------------


In [22]:
# open data/plwiktionary_definitions_clean.txt
with open('data/zagadki/plwiktionary_definitions_clean.txt', 'r') as file:
    final_answer_set = set([line.split()[0].lower() for line in file.readlines()])
print("Loaded", len(final_answer_set), "answers from plwiktionary_definitions_clean.txt")
print("Sample:", list(final_answer_set)[:50])

Loaded 8085 answers from plwiktionary_definitions_clean.txt
Sample: ['księga', 'płot', 'styczność', 'przypuszczenie', 'barszcz', 'spekulacja', 'zanieczyszczenie', 'wit', 'opel', 'codzienność', 'temperatura', 'wszechświat', 'spółgłoska', 'substancja', 'dziennikarz', 'bastion', 'kopiec', 'bigos', 'stwierdzenie', 'egzemplarz', 'światełko', 'brykiet', 'cytadela', 'pomoc', 'wiersz', 'mięta', 'elektron', 'felietonista', 'infolinia', 'kwestionariusz', 'pani', 'zaleta', 'montaż', 'rondo', 'sprzedawca', 'wyraz', 'obiektywność', 'kontrolowanie', 'darczyńca', 'bunkier', 'sekret', 'aktywizowanie', 'przyimek', 'szyja', 'ranek', 'amplituda', 'okładka', 'spichrz', 'marazm', 'czystość']


In [23]:
# zagadki_do_testow_clean.txt
with open('data/zagadki/zagadki_do_testow_clean.txt', 'r') as file:
    final_riddles = {line.split()[0]:' '.join(line.split()[2:]) for line in file.readlines()}

print("Loaded", len(final_riddles), "riddles from zagadki_do_testow_clean.txt")
print("Sample:", '\n'.join(map(str,list(final_riddles.items())[:5])))

true_answers = []
riddles_txt = []
for riddle, answer in final_riddles.items():
    riddles_txt.append(riddle)
    true_answers.append(answer)

Loaded 1993 riddles from zagadki_do_testow_clean.txt
Sample: ('manuskrypt', 'rękopiśmienny tekst lub dokument, niepublikowany drukiem.')
('wesołość', 'stan emocjonalny charakteryzujący się radością, życzliwością i łatwością w wywoływaniu uśmiechu.')
('legenda', 'opowieść lub historia przekazywana ustnie lub pisaną, często ze zmyślonymi lub niepotwierdzonymi elementami, która ma na celu przekazanie ważnych wartości, nauk, czy też przekonań.')
('antysemityzm', 'postawa, przekonania lub działania mające na celu dyskryminację, prześladowanie lub nienawiść wobec żydów jako grupy etnicznej, religijnej lub kulturowej.')
('filmowanie', 'proces rejestracji obrazu i dźwięku za pomocą kamery w celu tworzenia filmu.')


In [27]:
few_shot_examples = [
    ("kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem", "pasażerka"),
    ("emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu", "miłość"),
    ("suchą częścią powierzchni ziemi niezależną od akwenów wodnych.","ląd"),
    ("nieuprawnione i bezprawne wtargnięcie na czyjeś tereny lub do cudzego pomieszczenia w celu kradzieży lub popełnienia innych przestępstw.","włamanie"),
    ("potoczne określenie osoby nadużywającej alkoholu.","pijak"),
    ("proces testowania, badania lub wdrażania konkretnego rozwiązania, pomysłu lub projektu w praktyce.","pilotaż"),
    ("koncepcja związana z hinduizmem, buddyzmem i dżinizmem, mówiąca o zależności między działaniami jednostki a ich konsekwencjami w obecnym lub przyszłym życiu.","karma"),
    ("osoba zajmująca się badaniem i interpretacją przeszłości na podstawie dostępnych źródeł historycznych. może zajmować się różnymi epokami, wydarzeniami lub postaciami historycznymi.","historyk"),
    ("naśladowanie lub próba odtworzenia zachowania, stylu, charakterystyk lub cech innych osób lub rzeczy.","imitacja"),
    ("dodatek do ubioru zakładany na szyję lub ramiona, mający różne kształty, wzory i materiały.","szal")
]
from tqdm import tqdm
total_score = 0
iter = 0
# create empty file
open('data/zagadki/results.txt', 'w').close()
for true_answer, riddle_text in tqdm(list(final_riddles.items())[:100]):
    iter += 1
    # Build a few-shot prompt
    prompt = build_few_shot_prompt(riddle_text, examples=few_shot_examples)
    
    # Generate constrained answers
    answers = []
    for _ in range(3):
        answer = constrained_generate_answer(prompt, final_answer_set, max_new_tokens=10, temperature=0.7)
        answers.append(answer)

    correct = true_answer in answers
    total_score += correct
    print("Riddle:", riddle_text)
    print("Answer:", answers)
    print("Correct Answer:", true_answer)
    print("Correct?", correct)
    print(f"Score: {total_score}/{iter} - {total_score/iter:.2%}")
    print("-"*50)
    
    logs = "Riddle: " + riddle_text + "\n" + "Answer: " + str(answers) + "\n" + "Correct Answer: " + true_answer + "\n" + "Correct? " + str(correct) + "\n" + "Score: " + str(total_score) + "/" + str(iter) + " - " + str(total_score/iter) + "\n" + "-"*50 + "\n"
    with open('data/zagadki/results.txt', 'a') as file:
        file.write(logs)


  1%|          | 1/100 [07:13<11:54:50, 433.24s/it]

Riddle: rękopiśmienny tekst lub dokument, niepublikowany drukiem.
Answer: ['list', 'pamiętnik', 'notatka']
Correct Answer: manuskrypt
Correct? False
Score: 0/1 - 0.00%
--------------------------------------------------


  2%|▏         | 2/100 [17:42<14:56:06, 548.64s/it]

Riddle: stan emocjonalny charakteryzujący się radością, życzliwością i łatwością w wywoływaniu uśmiechu.
Answer: ['optymizm', 'euforia', 'euforia']
Correct Answer: wesołość
Correct? False
Score: 0/2 - 0.00%
--------------------------------------------------


  3%|▎         | 3/100 [26:42<14:40:19, 544.53s/it]

...

Riddle: aktywność polegająca na zanurzaniu się pod wodę, zazwyczaj za pomocą sprzętu oddechowego, w celu eksploracji podwodnego świata.
Answer: ['płetw', 'nur', 'nur']
Correct Answer: nurkowanie
Correct? False
Score: 1/6 - 16.67%
--------------------------------------------------


  7%|▋         | 7/100 [1:19:42<20:38:02, 798.74s/it]

Riddle: treść, wypowiedź lub zachowanie pozbawione sensu, logicznego uzasadnienia lub sensownego znaczenia.
Answer: ['deklaracja', 'przyzwoitość', 'tande']
Correct Answer: nonsens
Correct? False
Score: 1/7 - 14.29%
--------------------------------------------------


  8%|▊         | 8/100 [1:29:01<18:28:08, 722.70s/it]

Riddle: formalne odmówienie zgody na podjęcie określonej decyzji lub działania, stosowane w celu zablokowania lub opóźnienia procesu decyzyjnego.
Answer: ['rezygnacja', 'odmowa', 'zarząd']
Correct Answer: weto
Correct? False
Score: 1/8 - 12.50%
--------------------------------------------------


In [20]:
import ast

with open('task_2_results.txt', 'r') as file:
    content = '\n'.join(file.readlines())
    
riddles = content.split('--------------------------------------------------')
# Riddle: rękopiśmienny tekst lub dokument, niepublikowany drukiem.
# Answer: ['pieśń', 'pieśń', 'list', 'tek', 'apel']
# Correct Answer: manuskrypt
# Correct? False
# Score: 0/1 - 0.0
score = 0
for riddle in riddles:
    if not riddle:
        print(riddle)
        continue
    # get answer
    # print(riddle)
    answer = riddle.split('Answer: ')[1].split('\n')[0]
    if not answer or answer == '[]':
        print(riddle)
        continue
    correct_answer = riddle.split('Correct Answer: ')[1].split('\n')[0]
    for i, ans in enumerate(ast.literal_eval(answer)):
        if correct_answer in ans:
            score += 1/(i+1)
            break
print(f"Final Score: {score}/{len(riddles)} - {score/len(riddles):.2%}")
    
    
    
    

Final Score: 23.23333333333333/100 - 23.23%
